In [10]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader

In [11]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]
NUM_EPOCHS = 30
LEARNING_RATE = 1e-3

In [12]:
train_transform = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),       
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

In [13]:
train_dataset_path = '../../../dataset/train'
val_dataset_path = '../../../dataset/val'

train_dataset = datasets.ImageFolder(train_dataset_path, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dataset_path, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

In [14]:
model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
features = model.features
features.requires_grad_(False)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [15]:
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(4096, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [17]:
def train_one_epoch(model, device, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backwards()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return running_loss/total, correct/total

In [18]:
def validate(model, device, loader, criterion):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        val_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return val_loss/total, correct/total

In [19]:
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(model, device, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, device, val_loader, criterion)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}: "
          f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val loss {val_loss:.4f}, acc {train_acc:.4f}"
          )
    
    if(val_loss < best_val_loss):
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_vgg19.pth')

Total batches: 290
Epoch 1/30: Train loss 1.6266, acc 0.5158 | Val   loss 1.0063, acc 0.6737
Epoch 2/30: Train loss 1.5355, acc 0.6280 | Val   loss 0.7228, acc 0.7444
Epoch 3/30: Train loss 1.2184, acc 0.6848 | Val   loss 0.5452, acc 0.8202
Epoch 4/30: Train loss 1.1658, acc 0.6993 | Val   loss 0.6305, acc 0.7980
Epoch 5/30: Train loss 0.9405, acc 0.7410 | Val   loss 0.5267, acc 0.8343
Epoch 6/30: Train loss 0.9961, acc 0.7436 | Val   loss 0.5511, acc 0.8394
Epoch 7/30: Train loss 1.0090, acc 0.7374 | Val   loss 0.5888, acc 0.8253
Epoch 8/30: Train loss 0.9366, acc 0.7570 | Val   loss 0.6123, acc 0.8283
Epoch 9/30: Train loss 0.9219, acc 0.7575 | Val   loss 0.5533, acc 0.8364
Epoch 10/30: Train loss 0.8685, acc 0.7722 | Val   loss 0.4980, acc 0.8677
Epoch 11/30: Train loss 0.8231, acc 0.7730 | Val   loss 0.5362, acc 0.8535
Epoch 12/30: Train loss 0.8917, acc 0.7646 | Val   loss 0.4741, acc 0.8586
Epoch 13/30: Train loss 0.8045, acc 0.7823 | Val   loss 0.5381, acc 0.8606
Epoch 14/30: Tr